In [27]:
import sys
import os
import numpy as np
from tqdm import tqdm
import datetime

In [3]:
# vor_path = "/home/ubuntu/git/active_vertex"
vor_path = 'C:\\Users\\Pranav\\git\\active_vertex'
sys.path.append(vor_path)

import voronoi_model.voronoi_model_periodic as avm

In [42]:
p_space = np.linspace(3.5, 3.9, 5)
J_space = [0]
v_space = np.logspace(-2, 0, 5)[1:]

param_space = np.meshgrid(
    p_space, 
    J_space,
    v_space,
)
param_space = np.array(param_space).T.reshape(-1, 3)

In [43]:
param_space.shape

(20, 3)

In [44]:
f = 200
t0 = 0
tmax = 4
dt = 0.05
n_t = int((tmax - t0) * f / dt) + 1  # calculates the n_t to get the desired dt

a = 0.35
k = 2

In [47]:
param_space = np.array([
    [3.8, 0, 0.2],
    [3.5, 0, 0.2]
])

In [48]:
iterator = param_space
iterator = tqdm(iterator)

for p, J, v in iterator:
    prefix = f"p0{p:.2f}_J{J:.2f}_v0{v:.2e}"

    vor2 = avm.Tissue()
    vor2.generate_cells(600)
    vor2.make_init(10)

    vor2.set_GRN_t_span(t0, tmax, n_t, scaling_factor=f);
    vor2.v0 = v / f
    vor2.n_warmup_steps = int(200 / dt)

    W = J * np.array([[1, 0], [0, 1]])
    vor2.set_interaction(W=W, pE=0)
    vor2.A0 = 0.86
    vor2.P0 = p * np.sqrt(vor2.A0)

    vor2.Dr = 0.01
    vor2.kappa_A = 0.2
    vor2.kappa_P = 0.1
    vor2.a = a
    vor2.k = k

    vor2.simulate2(progress_bar=False, print_updates=False);

    vor2.animate(file_name=prefix, n_frames=100, print_updates=False)

100%|██████████| 2/2 [03:20<00:00, 100.09s/it]


<hr>